# Used Libraries

In [ ]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from typing import Optional
from pydantic import BaseModel


from fastapi.encoders import jsonable_encoder
from fastapi import FastAPI
from enum import Enum
import os
import io
import pickle
import uvicorn
import nest_asyncio

In [ ]:
def classify_tweet(tweets, model):
    '''
    The function used to classify tweets, but during to the model design it take a list of tweets,
    and return numpy array, so we have do some manipulation before return the data.
    
    Argument:
        tweets: dictionary of tweet text and default value = 0
        model: Enum of two string which the model you need to choose
    Return:
        classifed_tweet: dictionary of tweet text and default value = negative
    '''
    
    # get the tweet text as string then convert to list
    tweet = [tweets['tweet_text']]
    
    # Get the text features using Tf-Idf vectorization model
    tweets_features = tf_idf_model.transform(tweet)
    
    # convert to sparse matrix instead of compressed space type
    tweets_features_array = tweets_features.toarray()
    
    
    # Check which model you need to predicit 
    if model == "Logistic_Model":
        print("Prediction using Logistic Model")
        print("="*50)
        predict = log_reg_model.predict(tweets_features_array)
    else:
        print("Prediction using RandomForest Model")
        print("="*50)
        predict = randforest_model.predict(tweets_features_array)
        
    
    pred_result = 'Positive' if predict[0] == 1 else 'Negative'
    
    # Return dictionary
    classifed_tweet = {
        'tweet_text': tweet,
        'predict_class': pred_result
#         'predict_class': predict
    }
    return classifed_tweet

In [ ]:
def test_tweet(tweet_indx_with_100, model_type):
    test_tweet = list(covid_tweets_data['cleaned_tweet_text'])[tweet_indx_with_100]
    print(test_tweet)
    print("=============== True Value for this tweet ===================")
    print("=============== " + str(covid_tweets_data['class'][tweet_indx_with_100]) + " ===================")
    
    print("="*50)
    tweet_dict = {
        'tweet_text': test_tweet,
        'predict_class': '0'
    }
    
    return classify_tweet(tweet_dict, model_type)

In [ ]:
# Assign an instance of the FastAPI class to the variable "app".
# You will interact with your api using this instance.
app = FastAPI(title='Deploying a ML & DL Model with FastAPI')

# List available models using Enum
class Model(str, Enum):
    ML_Model = "Machine Learning Model"
    DL_Model = "Deep Learning Model"


class Tweet_Text(BaseModel):
    tweet_text: str = """ممثل منظمه الصحه العالميه في مصر يحدث في مصر اثق تماما في الاجراءات تتخذها مصر لمواجهه وباء محتمل MBCMASR """
    predict_class: str = 'IQ'
        

@app.get("/")
def home():
    # Once you go to this link you will see the get and post method below to trying out
    return "Congratulations! Your API is working as expected. Now head over to http://localhost:5000/docs."


# This endpoint handles all the logic necessary for the object detection to work.
# It requires the desired model and the dictionary of tweet and default class as we give default values to us
# In the api you can try other tweet from some_tweets below
@app.post("/predict") 
def prediction(model: Model, tweet: Tweet_Text):
    
    # Encode the retrived request data 
    tweet = jsonable_encoder(tweet)
    
    # Run our model
    classifed_tweet = classify_tweet(tweet, model=model)
    
    return classifed_tweet

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# uvicorn is fast Asynchronous Server Gateway Interface (ASGI) uvicorn handles the serving
# Spin up the server!    
uvicorn.run(app, host=host, port=5000)